In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# for notebook
%matplotlib inline

## Datasets

brazilian-soccer-database (libetadores) : 11 temporadas

In [41]:
# https://www.kaggle.com/datasets/ricardomattos05/brazilian-soccer-database

libertadores = pd.read_csv('../data/BrazilianSoccer/Data/Libertadores_Matches.csv', sep=',')
df = libertadores.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   1254 non-null   object 
 1   home_team  1255 non-null   object 
 2   away_team  1255 non-null   object 
 3   home_goal  1255 non-null   object 
 4   away_goal  1255 non-null   object 
 5   season     1254 non-null   float64
 6   stage      1255 non-null   object 
dtypes: float64(1), object(6)
memory usage: 68.8+ KB


In [42]:
df.head(20)

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2013-02-12 20:15:00,Nacional (URU),Barcelona-EQU,2,2,2013.0,group stage
1,2013-02-13 22:00:00,Boca Juniors,Toluca,1,2,2013.0,group stage
2,2013-02-19 23:45:00,Toluca,Nacional (URU),2,3,2013.0,group stage
3,2013-02-27 19:45:00,Barcelona-EQU,Boca Juniors,1,2,2013.0,group stage
4,2013-03-07 00:15:00,Toluca,Barcelona-EQU,1,1,2013.0,group stage
5,2013-03-07 21:30:00,Boca Juniors,Nacional (URU),0,1,2013.0,group stage
6,2013-03-13 19:45:00,Barcelona-EQU,Toluca,0,0,2013.0,group stage
7,2013-03-14 19:15:00,Nacional (URU),Boca Juniors,0,1,2013.0,group stage
8,2013-04-03 19:45:00,Boca Juniors,Barcelona-EQU,1,0,2013.0,group stage
9,2013-04-04 19:15:00,Nacional (URU),Toluca,4,0,2013.0,group stage


In [43]:
df.tail(20)

,datetime,home_team,away_team,home_goal,away_goal,season,stage
1235,2022-07-06 21:30:00,Flamengo,Tolima,7,1,2022.0,round of 16
1236,2022-06-28 21:30:00,Corinthians,Boca Juniors,0,0,2022.0,round of 16
1237,2022-07-05 21:30:00,Boca Juniors,Corinthians,0,0,2022.0,round of 16
1238,2022-06-29 19:15:00,Talleres,Colón,1,1,2022.0,round of 16
1239,2022-07-06 19:15:00,Colón,Talleres,0,2,2022.0,round of 16
1240,2022-06-29 21:30:00,Vélez Sarsfield,River Plate,1,0,2022.0,round of 16
1241,2022-07-06 21:30:00,River Plate,Vélez Sarsfield,0,0,2022.0,round of 16
1242,2022-08-04 21:30:00,Athletico,Estudiantes,0,0,2022.0,quarterfinals
1243,2022-08-11 21:30:00,Estudiantes,Athletico,0,1,2022.0,quarterfinals
1244,2022-08-03 21:30:00,Atlético-MG,Palmeiras,2,2,2022.0,quarterfinals


In [44]:
df.columns

Index(['datetime', 'home_team', 'away_team', 'home_goal', 'away_goal',
       'season', 'stage'],
      dtype='object')

In [45]:
df = df[['season', 'home_team', 'away_team', 'home_goal', 'away_goal']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   season     1254 non-null   float64
 1   home_team  1255 non-null   object 
 2   away_team  1255 non-null   object 
 3   home_goal  1255 non-null   object 
 4   away_goal  1255 non-null   object 
dtypes: float64(1), object(4)
memory usage: 49.1+ KB


In [46]:
df = df.dropna(how='any',axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 1253
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   season     1254 non-null   float64
 1   home_team  1254 non-null   object 
 2   away_team  1254 non-null   object 
 3   home_goal  1254 non-null   object 
 4   away_goal  1254 non-null   object 
dtypes: float64(1), object(4)
memory usage: 58.8+ KB


In [47]:
df.rename(columns = {
    'season':'year', 
    'home_team':'home', 
    'away_team':'away', 
    'home_goal':'home_goals', 
    'away_goal':'away_goals'
    }, inplace = True)

In [48]:
df['home_goals'] = pd.to_numeric(df['home_goals'], errors='coerce')
df['away_goals'] = pd.to_numeric(df['away_goals'], errors='coerce')
df['year'] = pd.to_numeric(df['year'], errors='coerce')

In [49]:
print(sorted(df['year'].unique()))
print("#temporadas = ", len(df['year'].unique()))

[2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0]
#temporadas =  10


In [50]:
#temp = ['2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']
#df = df.loc[df['year'].isin(temp)].reset_index()

In [51]:
vhome = list(df['home'].unique())
print(len(vhome))

vaway = list(df['away'].unique())
print(len(vaway))

vertice = set(vhome) | set(vaway)
print(len(vertice))
print(list(vertice))

114
114
114
['Deportivo Táchira', 'Unión Española', 'Zamora', 'Independiente del Valle', 'Defensor Sporting', 'Always Ready', 'River Plate', 'Universitario (PER)', 'Vélez Sarsfield', 'Independiente Del Valle', 'Universitario de Sucre', 'La Guaira', 'Ind. Santa Fe', 'Huachipato', 'Palestino', 'Emelec', 'Junior de Barranquilla', 'San Lorenzo', 'Deportivo Cali', 'Red Bull Bragantino', 'Rosario Central', 'Real Atlético', 'Estudiantes', 'Guaraní-PAR', 'São Paulo', 'Estudiantes de Mérida', 'Real Garcilaso', 'Tijuana', 'Jorge Wilstermann', 'Universidad Católica', 'Racing Club', 'Colón', 'Monagas', 'Cruzeiro', 'Arsenal Sarandí', 'Atlético Nacional', 'Lanús', 'Cerro Porteño', 'Trujillanos (VEN)', 'Melgar', 'Delfín', 'Sport Boys', 'Independiente', 'Santos', 'Huracán', 'Universidad de Concepción', 'Pumas', 'América de Cali', 'Nacional-URU', 'América-MG', 'Vasco', 'Caracas', 'Delfín-EQU', 'Alianza Lima', 'Atlético-PR', 'River Plate-URU', 'Corinthians', 'Argentinos Juniors', 'León', 'Santos Laguna'

In [53]:
df['home_win'] = 0.0
df['away_win'] = 0.0
df['draw'] = 0.0

df.loc[df['home_goals'] > df['away_goals'], 'home_win'] = 1.0
df.loc[df['home_goals'] < df['away_goals'], 'away_win'] = 1.0
df.loc[df['home_goals'] == df['away_goals'], 'draw'] = 1.0

df.head(20)

,year,home,away,home_goals,away_goals,home_win,away_win,draw
0,2013.0,Nacional (URU),Barcelona-EQU,2.0,2.0,0.0,0.0,1.0
1,2013.0,Boca Juniors,Toluca,1.0,2.0,0.0,1.0,0.0
2,2013.0,Toluca,Nacional (URU),2.0,3.0,0.0,1.0,0.0
3,2013.0,Barcelona-EQU,Boca Juniors,1.0,2.0,0.0,1.0,0.0
4,2013.0,Toluca,Barcelona-EQU,1.0,1.0,0.0,0.0,1.0
5,2013.0,Boca Juniors,Nacional (URU),0.0,1.0,0.0,1.0,0.0
6,2013.0,Barcelona-EQU,Toluca,0.0,0.0,0.0,0.0,1.0
7,2013.0,Nacional (URU),Boca Juniors,0.0,1.0,0.0,1.0,0.0
8,2013.0,Boca Juniors,Barcelona-EQU,1.0,0.0,1.0,0.0,0.0
9,2013.0,Nacional (URU),Toluca,4.0,0.0,1.0,0.0,0.0


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 1253
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1254 non-null   float64
 1   home        1254 non-null   object 
 2   away        1254 non-null   object 
 3   home_goals  1253 non-null   float64
 4   away_goals  1253 non-null   float64
 5   home_win    1254 non-null   float64
 6   away_win    1254 non-null   float64
 7   draw        1254 non-null   float64
dtypes: float64(6), object(2)
memory usage: 88.2+ KB


In [55]:
df2 = df.groupby(
    ['home', 'away'],
    as_index=False,
    sort=False).agg({'home_goals':'sum', 'away_goals':'sum', 'home_win':'sum', 'away_win':'sum', 'draw' : 'sum'})

df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Nacional (URU),Barcelona-EQU,2.0,2.0,0.0,0.0,1.0
1,Boca Juniors,Toluca,1.0,2.0,0.0,1.0,0.0
2,Toluca,Nacional (URU),2.0,3.0,0.0,1.0,0.0
3,Barcelona-EQU,Boca Juniors,2.0,2.0,1.0,1.0,0.0
4,Toluca,Barcelona-EQU,1.0,1.0,0.0,0.0,1.0
5,Boca Juniors,Nacional (URU),1.0,2.0,0.0,1.0,1.0
6,Barcelona-EQU,Toluca,0.0,0.0,0.0,0.0,1.0
7,Nacional (URU),Boca Juniors,1.0,2.0,0.0,1.0,1.0
8,Boca Juniors,Barcelona-EQU,1.0,0.0,1.0,0.0,1.0
9,Nacional (URU),Toluca,4.0,0.0,1.0,0.0,0.0


In [56]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        1125 non-null   object 
 1   away        1125 non-null   object 
 2   home_goals  1125 non-null   float64
 3   away_goals  1125 non-null   float64
 4   home_win    1125 non-null   float64
 5   away_win    1125 non-null   float64
 6   draw        1125 non-null   float64
dtypes: float64(5), object(2)
memory usage: 61.6+ KB


In [57]:
lista = []
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away
    t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away']]
    
    if len(t) == 0:
        lista.append(time1)
        print(time2,time1)

print(len(lista))

Santos Palmeiras
1


In [58]:
time1 = "Santos"
time2 = "Palmeiras"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw']]
print(t.values)

[]
[['Palmeiras' 'Santos' 1.0 0.0 1.0 0.0 0.0]]


In [59]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
0,Nacional (URU),Barcelona-EQU,2.0,2.0,0.0,0.0,1.0
1,Boca Juniors,Toluca,1.0,2.0,0.0,1.0,0.0
2,Toluca,Nacional (URU),2.0,3.0,0.0,1.0,0.0
3,Barcelona-EQU,Boca Juniors,2.0,2.0,1.0,1.0,0.0
4,Toluca,Barcelona-EQU,1.0,1.0,0.0,0.0,1.0
5,Boca Juniors,Nacional (URU),1.0,2.0,0.0,1.0,1.0
6,Barcelona-EQU,Toluca,0.0,0.0,0.0,0.0,1.0
7,Nacional (URU),Boca Juniors,1.0,2.0,0.0,1.0,1.0
8,Boca Juniors,Barcelona-EQU,1.0,0.0,1.0,0.0,1.0
9,Nacional (URU),Toluca,4.0,0.0,1.0,0.0,0.0


In [60]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw
1105,Flamengo,Sporting Cristal,2.0,1.0,1.0,0.0,0.0
1106,Universidad Católica,Talleres,0.0,1.0,0.0,1.0,0.0
1107,Fortaleza,Estudiantes,1.0,1.0,0.0,0.0,1.0
1108,Estudiantes,Fortaleza,3.0,0.0,1.0,0.0,0.0
1109,Emelec,Atlético-MG,1.0,1.0,0.0,0.0,1.0
1110,Atlético-MG,Emelec,1.0,0.0,1.0,0.0,0.0
1111,Tolima,Flamengo,0.0,1.0,0.0,1.0,0.0
1112,Flamengo,Tolima,7.0,1.0,1.0,0.0,0.0
1113,Talleres,Colón,1.0,1.0,0.0,0.0,1.0
1114,Colón,Talleres,0.0,2.0,0.0,1.0,0.0


In [61]:
df2["weight1"] = 0.0
df2["weight2"] = 0.0

In [62]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['away_goals','away_win','draw']].values
    t2 =  df2.loc[(df2['away'] == time1) & (df2['home'] == time2)][['home_goals','home_win','draw']].values

    g1 = p1 = 0
    for i1 in t1:
        g1 = i1[0]
        p1 = 3*i1[1] + 1*i1[2]

    g2 = p2 = 0
    for i2 in t2:
        g2 = i2[0]
        p2 = 3*i2[1] + 1*i2[2]

    df2.loc[keys,'weight1'] = g1 + g2
    df2.loc[keys,'weight2'] = p1 + p2

In [63]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Nacional (URU),Barcelona-EQU,2.0,2.0,0.0,0.0,1.0,3.0,4.0
1,Boca Juniors,Toluca,1.0,2.0,0.0,1.0,0.0,5.0,6.0
2,Toluca,Nacional (URU),2.0,3.0,0.0,1.0,0.0,7.0,6.0
3,Barcelona-EQU,Boca Juniors,2.0,2.0,1.0,1.0,0.0,3.0,7.0
4,Toluca,Barcelona-EQU,1.0,1.0,0.0,0.0,1.0,1.0,2.0
5,Boca Juniors,Nacional (URU),1.0,2.0,0.0,1.0,1.0,3.0,5.0
6,Barcelona-EQU,Toluca,0.0,0.0,0.0,0.0,1.0,1.0,2.0
7,Nacional (URU),Boca Juniors,1.0,2.0,0.0,1.0,1.0,3.0,5.0
8,Boca Juniors,Barcelona-EQU,1.0,0.0,1.0,0.0,1.0,2.0,4.0
9,Nacional (URU),Toluca,4.0,0.0,1.0,0.0,0.0,2.0,0.0


In [64]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
1105,Flamengo,Sporting Cristal,2.0,1.0,1.0,0.0,0.0,1.0,0.0
1106,Universidad Católica,Talleres,0.0,1.0,0.0,1.0,0.0,2.0,6.0
1107,Fortaleza,Estudiantes,1.0,1.0,0.0,0.0,1.0,4.0,4.0
1108,Estudiantes,Fortaleza,3.0,0.0,1.0,0.0,0.0,1.0,1.0
1109,Emelec,Atlético-MG,1.0,1.0,0.0,0.0,1.0,2.0,4.0
1110,Atlético-MG,Emelec,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1111,Tolima,Flamengo,0.0,1.0,0.0,1.0,0.0,8.0,6.0
1112,Flamengo,Tolima,7.0,1.0,1.0,0.0,0.0,1.0,0.0
1113,Talleres,Colón,1.0,1.0,0.0,0.0,1.0,1.0,1.0
1114,Colón,Talleres,0.0,2.0,0.0,1.0,0.0,3.0,4.0


In [65]:
for keys, infos in df2.iterrows():
    time1 = infos.home
    time2 = infos.away

    t2 = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)]
    
    if len(t2) == 0:
        t1 =  df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home_goals','home_win','draw']].values

        w1 = w2 = 0
        for i1 in t1:
            w1 = i1[0]
            w2 = 3*i1[1] + 1*i1[2]

        new = [time2, time1,np.nan,np.nan,0,0,0,w1,w2]

        new = pd.DataFrame(columns=df2.columns, data=[new])
        
        df2 = pd.concat([df2, new], axis=0)

In [66]:
df2.head(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
0,Nacional (URU),Barcelona-EQU,2.0,2.0,0.0,0.0,1.0,3.0,4.0
1,Boca Juniors,Toluca,1.0,2.0,0.0,1.0,0.0,5.0,6.0
2,Toluca,Nacional (URU),2.0,3.0,0.0,1.0,0.0,7.0,6.0
3,Barcelona-EQU,Boca Juniors,2.0,2.0,1.0,1.0,0.0,3.0,7.0
4,Toluca,Barcelona-EQU,1.0,1.0,0.0,0.0,1.0,1.0,2.0
5,Boca Juniors,Nacional (URU),1.0,2.0,0.0,1.0,1.0,3.0,5.0
6,Barcelona-EQU,Toluca,0.0,0.0,0.0,0.0,1.0,1.0,2.0
7,Nacional (URU),Boca Juniors,1.0,2.0,0.0,1.0,1.0,3.0,5.0
8,Boca Juniors,Barcelona-EQU,1.0,0.0,1.0,0.0,1.0,2.0,4.0
9,Nacional (URU),Toluca,4.0,0.0,1.0,0.0,0.0,2.0,0.0


In [67]:
df2.tail(20)

,home,away,home_goals,away_goals,home_win,away_win,draw,weight1,weight2
1106,Universidad Católica,Talleres,0.0,1.0,0.0,1.0,0.0,2.0,6.0
1107,Fortaleza,Estudiantes,1.0,1.0,0.0,0.0,1.0,4.0,4.0
1108,Estudiantes,Fortaleza,3.0,0.0,1.0,0.0,0.0,1.0,1.0
1109,Emelec,Atlético-MG,1.0,1.0,0.0,0.0,1.0,2.0,4.0
1110,Atlético-MG,Emelec,1.0,0.0,1.0,0.0,0.0,1.0,1.0
1111,Tolima,Flamengo,0.0,1.0,0.0,1.0,0.0,8.0,6.0
1112,Flamengo,Tolima,7.0,1.0,1.0,0.0,0.0,1.0,0.0
1113,Talleres,Colón,1.0,1.0,0.0,0.0,1.0,1.0,1.0
1114,Colón,Talleres,0.0,2.0,0.0,1.0,0.0,3.0,4.0
1115,Vélez Sarsfield,River Plate,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [68]:
time1 = "Santos"
time2 = "Palmeiras"

t = df2.loc[(df2['home'] == time1) & (df2['away'] == time2)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

t = df2.loc[(df2['home'] == time2) & (df2['away'] == time1)][['home','away','home_goals','away_goals','home_win', 'away_win', 'draw','weight1','weight2']]
print(t.values)

[['Santos' 'Palmeiras' nan nan 0.0 0.0 0.0 1.0 3.0]]
[['Palmeiras' 'Santos' 1.0 0.0 1.0 0.0 0.0 0.0 0.0]]


In [69]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   home        1126 non-null   object 
 1   away        1126 non-null   object 
 2   home_goals  1125 non-null   float64
 3   away_goals  1125 non-null   float64
 4   home_win    1126 non-null   float64
 5   away_win    1126 non-null   float64
 6   draw        1126 non-null   float64
 7   weight1     1126 non-null   float64
 8   weight2     1126 non-null   float64
dtypes: float64(7), object(2)
memory usage: 88.0+ KB


In [70]:
df3 = df2[['home','away','weight1','weight2']]

df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126 entries, 0 to 0
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   home     1126 non-null   object 
 1   away     1126 non-null   object 
 2   weight1  1126 non-null   float64
 3   weight2  1126 non-null   float64
dtypes: float64(2), object(2)
memory usage: 44.0+ KB


In [71]:
df3.head(20)

,home,away,weight1,weight2
0,Nacional (URU),Barcelona-EQU,3.0,4.0
1,Boca Juniors,Toluca,5.0,6.0
2,Toluca,Nacional (URU),7.0,6.0
3,Barcelona-EQU,Boca Juniors,3.0,7.0
4,Toluca,Barcelona-EQU,1.0,2.0
5,Boca Juniors,Nacional (URU),3.0,5.0
6,Barcelona-EQU,Toluca,1.0,2.0
7,Nacional (URU),Boca Juniors,3.0,5.0
8,Boca Juniors,Barcelona-EQU,2.0,4.0
9,Nacional (URU),Toluca,2.0,0.0


In [72]:
df3.tail(20)

,home,away,weight1,weight2
1106,Universidad Católica,Talleres,2.0,6.0
1107,Fortaleza,Estudiantes,4.0,4.0
1108,Estudiantes,Fortaleza,1.0,1.0
1109,Emelec,Atlético-MG,2.0,4.0
1110,Atlético-MG,Emelec,1.0,1.0
1111,Tolima,Flamengo,8.0,6.0
1112,Flamengo,Tolima,1.0,0.0
1113,Talleres,Colón,1.0,1.0
1114,Colón,Talleres,3.0,4.0
1115,Vélez Sarsfield,River Plate,0.0,1.0


In [73]:
df3.to_csv('../data/work1/libertadores_2013_2022.csv', index=False, sep=';')